In [1]:
import numpy as np
import os
import time
folder = "Problems"
time_limit = 300

In [2]:
def get_all_mst(distances, start_time, time_limit):
    n = len(distances)
    data = [0]*2**(n-1)
    cur = 0
    is_used = [False]*n
    def dfs(cur, state, value, rest_locations, cost):
        data[state] = value
        m = len(rest_locations)
        if m == 0:
            return
        for i in rest_locations:
            cost[i] = min(cost[i], distances[i+1][cur+1])
        rest_locations.sort(key = lambda x: cost[x], reverse = True)
        for _ in range(m):
            if time.time() - start_time > time_limit:
                return
            next = rest_locations.pop()
            
            dfs(next, state + 2**next, value + cost[next],rest_locations[:], cost[:])
    dfs(-1, 0, 0, list(np.arange(n-1)), [np.inf]*(n-1))
    return data

In [3]:
result = [[] for _ in os.listdir(folder)]
is_terminate = [False for _ in os.listdir(folder)]
for step, dirs in enumerate(sorted(os.listdir(folder), key = lambda x: int(x))):
    print("Test with number of {} begin!".format(dirs))
    cur_dir = os.path.join(folder, dirs)
    count = 0
    for files in os.listdir(cur_dir):
        start_time = time.time()
        path = os.path.join(cur_dir, files)
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read().split()
            data = list(map(float, content))
            number = int(data[0])
            distances = np.zeros((number,number))
        for i in range(number):
            for j in range(number):
                distances[i][j] = data[i*number+j+1]
        distances_order = [sorted(np.arange(number), key = lambda x: distances[x][i]) for i in range(number)]
        
        data = get_all_mst(distances, start_time, time_limit/2)
    
        Is_been_to = [False]*number
        Is_been_to[0] = True
        cur_route = [0]
        ans = [np.inf]
        ans_route = [""]
        terminate = [False]
        def dfs(cur, step, cur_sum, state):

            if time.time() - start_time > time_limit:
                terminate[0] = True
            if step == number-1:
                
                if ans[0] > cur_sum + distances[cur][0]:
                    ans[0] = cur_sum + distances[cur][0]
                    cur_route.append(0)
                    ans_route[0] = "->".join(map(str, cur_route))
                    cur_route.pop()
                return
            
            
            Is_been_to[cur] = True
            if cur != 0:
                state = state ^ (1 << (cur-1))
            hubrist = data[state]
            for i in distances_order[cur]:
                if terminate[0]:
                    return 
                if Is_been_to[i] == False and cur_sum+distances[cur][i] + hubrist < ans[0]:
                    
                    cur_route.append(i)
                    dfs(i, step+1, cur_sum+distances[cur][i], state)
                    
                    cur_route.pop() 
            Is_been_to[cur] = False
        dfs(0,0,0,2**(number-1)-1)
        print("ans is: {}, route is: {}".format(np.round(ans[0], 2), ans_route[0]))
        result[step].append(ans[0])
    print()
    print()
    


Test with number of 10 begin!
ans is: 469.78, route is: 0->6->5->4->2->8->3->9->1->7->0
ans is: 449.22, route is: 0->5->4->2->7->9->3->8->6->1->0
ans is: 416.75, route is: 0->2->4->6->8->5->7->1->9->3->0
ans is: 506.28, route is: 0->2->6->4->5->7->1->3->8->9->0
ans is: 518.05, route is: 0->9->2->8->5->1->3->4->7->6->0
ans is: 420.27, route is: 0->3->7->6->5->9->4->8->2->1->0
ans is: 547.28, route is: 0->7->6->1->4->2->8->3->5->9->0
ans is: 484.33, route is: 0->2->6->9->8->1->4->3->5->7->0
ans is: 735.16, route is: 0->6->9->8->5->4->7->3->2->1->0
ans is: 470.02, route is: 0->8->1->6->4->5->3->9->7->2->0


Test with number of 20 begin!
ans is: 734.74, route is: 0->12->7->11->3->2->19->5->14->8->18->9->16->6->13->1->15->17->10->4->0
ans is: 638.5, route is: 0->8->1->19->6->4->15->9->3->13->17->10->7->2->18->14->5->16->12->11->0
ans is: 733.38, route is: 0->9->4->8->10->16->5->2->15->1->17->6->14->19->3->13->11->18->12->7->0
ans is: 455.43, route is: 0->16->11->9->8->12->18->1->6->13->10->

MemoryError: 

In [6]:
count

10

In [5]:
from openpyxl import load_workbook

# 载入已存在的xlsx文件
wb = load_workbook('result.xlsx')

# 选择一个工作表
ws = wb["Accurate"]

# 写入数据到第A列（你可以更改列名来选择不同的列）
ws[f'A{1}'] = "MST2"
for i, item in enumerate(result, start=2):  # start=1 表示从第一行开始
    ws[f'A{i}'] = np.mean(item)
# 保存文件
wb.save('result.xlsx')